In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import os
import time
import requests
import csv
import pandas as pd
import matplotlib.pyplot as plt

In [56]:
# Config

# Create Backup
# --------------------

os.system("mkdir backup")

0

In [89]:
## Richemont Scrapper

richemont_job_db = []

browser = webdriver.Chrome('/Users/maxime_alain/Downloads/chromedriver')
browser.get('https://jobs.richemont.com/search/?q=&sortColumn=referencedate&sortDirection=desc&optionsFacetsDD_country=FR&startrow=1')

soup = BeautifulSoup(browser.page_source,'lxml')

for x in soup.find_all('tr', attrs={'class' : 'data-row clickable'}):
    row_scrap = []
    job_title = x.find('span', attrs={'class' : 'jobTitle'}).text.replace('\n', '')
    job_link = x.find('span', attrs={'class' : 'jobTitle'}).find('a')['href']
    company = x.find('span', attrs={'class' : 'jobFacility'}).text.replace('\n', '')
    departement = x.find('span', attrs={'class' : 'jobDepartment'}).text.replace('\n', '')
    location = x.find('span', attrs={'class' : 'jobLocation'}).text.replace('\n', '').strip()
    
    row_scrap.append(job_title)
    row_scrap.append(job_link)
    row_scrap.append(company)
    row_scrap.append(departement)
    row_scrap.append(location)
    
    richemont_job_db.append(row_scrap)

i = 14
while i <= int(soup.find('a', class_='paginationItemLast')['href'][-3:]) :
    browser.get(f'https://jobs.richemont.com/search/?q=&sortColumn=referencedate&sortDirection=desc&optionsFacetsDD_country=FR&startrow={i}')

    soup = BeautifulSoup(browser.page_source,'lxml')
    
    for x in soup.find_all('tr', attrs={'class' : 'data-row clickable'}):
        row_scrap = []
        job_title = x.find('span', attrs={'class' : 'jobTitle'}).text.replace('\n', '')
        job_link = x.find('span', attrs={'class' : 'jobTitle'}).find('a')['href']
        company = x.find('span', attrs={'class' : 'jobFacility'}).text.replace('\n', '')
        departement = x.find('span', attrs={'class' : 'jobDepartment'}).text.replace('\n', '')
        location = x.find('span', attrs={'class' : 'jobLocation'}).text.replace('\n', '').strip()

        row_scrap.append(job_title)
        row_scrap.append(job_link)
        row_scrap.append(company)
        row_scrap.append(departement)
        row_scrap.append(location)

        richemont_job_db.append(row_scrap)
    
    i = i + 14
    
richemont_job_db = pd.DataFrame(richemont_job_db)
richemont_job_db = richemont_job_db.rename(columns={0 : "Position", 1 : "link", 2 : "Company",
                                                    3 : "Department", 4 : "Location"})

richemont_job_db_copy = richemont_job_db
richemont_job_db_copy['desc'] = ''

# Add Desc to job offer
# add https://jobs.richemont.com before links to connect

i = 0

while i <= len(richemont_job_db_copy)-1:
    browser.get('https://jobs.richemont.com' + richemont_job_db_copy['link'][i])
    soup = BeautifulSoup(browser.page_source,'lxml')
    richemont_job_db_copy['desc'][i] = soup.find('span', class_="jobdescription").text
    i = i + 1
richemont_job_db_copy.head()

richemont_job_db_copy.to_csv('backup/richemont_db.csv')
richemont_job_db_copy.to_json('backup/richemont_db.json')

In [91]:
## Dassault Aviation

dassaultAviation_db = []

browser = webdriver.Chrome('/Users/maxime_alain/Downloads/chromedriver')

i = 1
while i <= 10:
    browser.get(f'https://carriere.dassault-aviation.com/offre-de-emploi/liste-offres.aspx?page={i}&LCID=1036')
    soup = BeautifulSoup(browser.page_source,'lxml')

    for x in soup.find_all('li', class_='ts-offer-list-item'):
            row_scrap = []
            job_title = x.find('a', attrs={'class' : 'ts-offer-list-item__title-link'}).text.replace('\n', '').strip()
            job_link = x.find('a', attrs={'class' : 'ts-offer-list-item__title-link'})['href']
            company = "Dassault Aviation"
            departement = 'N/A'
            details = x.find('ul', attrs={'class' : 'ts-offer-list-item__description'}).find_all('li')

            row_scrap.append(job_title)
            row_scrap.append(job_link)
            row_scrap.append(company)
            row_scrap.append(departement)
            row_scrap.append(details)

            dassaultAviation_db.append(row_scrap)
    
    i = i + 1

dassaultAviation_db = pd.DataFrame(dassaultAviation_db)
dassaultAviation_db = dassaultAviation_db.rename(columns={0 : "Position", 1 : "link", 2 : "Company", 3 : "Department", 4 : "Location"}) #
dassaultAviation_db_copy = dassaultAviation_db
dassaultAviation_db_copy['desc'] = ''

# Add Desc to job offer
# add https://carriere.dassault-aviation.com before links to connect

i = 0
while i <= len(dassaultAviation_db_copy)-1:
    browser.get('https://carriere.dassault-aviation.com' + dassaultAviation_db_copy['link'][i])
    soup = BeautifulSoup(browser.page_source,'lxml')
    dassaultAviation_db_copy['desc'][i] = soup.find('div', attrs={'id' : 'contenu-ficheoffre'}).text
    i = i + 1

dassaultAviation_db_copy.to_csv('backup/dassaultAviation_db.csv')
dassaultAviation_db_copy.head()

In [ ]:
# Air France

browser = webdriver.Chrome('/Users/maxime_alain/Downloads/chromedriver')

airFrance_db = []

browser.get('https://recrutement.airfrance.com/offre-de-emploi/liste-offres.aspx')
soup = BeautifulSoup(browser.page_source,'lxml')

for x in soup.find_all('li', class_='ts-offer-list-item'):
            row_scrap = []
            job_title = x.find('a', attrs={'class' : 'ts-offer-list-item__title-link'}).text.replace('\n', '').strip()
            job_link = x.find('a', attrs={'class' : 'ts-offer-list-item__title-link'})['href']
            company = "Air France"
            departement = 'N/A'
            details = x.find('ul', attrs={'class' : 'ts-offer-list-item__description'}).find_all('li')

            row_scrap.append(job_title)
            row_scrap.append(job_link)
            row_scrap.append(company)
            row_scrap.append(departement)
            row_scrap.append(details)

            airFrance_db.append(row_scrap)

airFrance_db = pd.DataFrame(airFrance_db)
airFrance_db = airFrance_db.rename(columns={0 : "Position", 1 : "link", 2 : "Company", 3 : "Department", 4 : "Location"}) #
airFrance_db_copy = airFrance_db
airFrance_db_copy['desc'] = ''

i = 0
while i <= len(airFrance_db_copy)-1:
    browser.get('https://recrutement.airfrance.com' + airFrance_db_copy['link'][i])
    soup = BeautifulSoup(browser.page_source,'lxml')
    airFrance_db_copy['desc'][i] = soup.find('div', attrs={'id' : 'contenu-ficheoffre'}).text
    i = i + 1

airFrance_db_copy.to_csv('backup/airFrance_db.csv')
airFrance_db_copy.head()

In [148]:
# Sanofi

browser = webdriver.Chrome('/Users/maxime_alain/Downloads/chromedriver')
browser.get('https://fr.jobs.sanofi.com/recherche-d%27offres/?p=1')
soup = BeautifulSoup(browser.page_source,'lxml')

sanofi_db = []

i = 1
while i <= int(soup.find('input', class_='pagination-current')['max']):
    browser.get(f'https://fr.jobs.sanofi.com/recherche-d%27offres/?p={i}')
    soup = BeautifulSoup(browser.page_source,'lxml')
    for x in soup.find('ul', class_='unstyled-list job-list').find_all('li'):
                row_scrap = []
                job_title = x.find('h2', attrs={'class' : 'h3-style'}).text.replace('\n', '').strip()
                job_link = x.find('a')['href']
                company = "Sanofi"
                departement = 'N/A'
                details = x.find('span', attrs={'class' : 'job-location'}).text

                row_scrap.append(job_title)
                row_scrap.append(job_link)
                row_scrap.append(company)
                row_scrap.append(departement)
                row_scrap.append(details)

                sanofi_db.append(row_scrap)
                
    i = i + 1

sanofi_db = pd.DataFrame(sanofi_db)
sanofi_db = sanofi_db.rename(columns={0 : "Position", 1 : "link", 2 : "Company", 3 : "Department", 4 : "Location"}) #
sanofi_db_copy = sanofi_db
sanofi_db_copy['desc'] = ''

i = 0
while i <= len(sanofi_db_copy)-1:
    browser.get('https://fr.jobs.sanofi.com/' + sanofi_db_copy['link'][i])
    soup = BeautifulSoup(browser.page_source,'lxml')
    try :
        sanofi_db_copy['desc'][i] = soup.find('div', attrs={'class' : 'ats-description'}).text
    except :
        sanofi_db_copy['desc'][i] = ''
    i = i + 1

sanofi_db_copy.to_csv('backup/sanofi_db.csv')
sanofi_db_copy.head()

,Position,link,Company,Department,Location,desc
0,Regulatory Site Officer External Supply QA,/emploi/waltham/regulatory-site-officer-extern...,Sanofi,N/A,"Waltham, Massachusetts",POSITION OVERVIEWRegulatory Site Officer Exter...
1,HSE Analyst,/emploi/ocoyoacac/hse-analyst/20874/18068424,Sanofi,N/A,"Ocoyoacac, Mexique",ANALISTA DE HSEOCOYOACACSanofi es uno de los p...
2,Maintenance Technician - Vaccine,/emploi/east-swiftwater/maintenance-technician...,Sanofi,N/A,"Swiftwater, PA - Lieux Multiples",POSITION OVERVIEW:Work Shift: This is a 2nd sh...
3,Medical Science Liaison - Hematologic Oncology...,/emploi/chicago/medical-science-liaison-hemato...,Sanofi,N/A,"Chicago, Illinois",The Mission of Sanofi’s MSL Team and the Medi...
4,Quality Control Manager,/emploi/waltham/quality-control-manager/20874/...,Sanofi,N/A,"Waltham, Massachusetts",External Supply Quality Control Manager(CLO ov...


In [149]:
sanofi_db_copy

,Position,link,Company,Department,Location,desc
0,Regulatory Site Officer External Supply QA,/emploi/waltham/regulatory-site-officer-extern...,Sanofi,N/A,"Waltham, Massachusetts",POSITION OVERVIEWRegulatory Site Officer Exter...
1,HSE Analyst,/emploi/ocoyoacac/hse-analyst/20874/18068424,Sanofi,N/A,"Ocoyoacac, Mexique",ANALISTA DE HSEOCOYOACACSanofi es uno de los p...
2,Maintenance Technician - Vaccine,/emploi/east-swiftwater/maintenance-technician...,Sanofi,N/A,"Swiftwater, PA - Lieux Multiples",POSITION OVERVIEW:Work Shift: This is a 2nd sh...
3,Medical Science Liaison - Hematologic Oncology...,/emploi/chicago/medical-science-liaison-hemato...,Sanofi,N/A,"Chicago, Illinois",The Mission of Sanofi’s MSL Team and the Medi...
4,Quality Control Manager,/emploi/waltham/quality-control-manager/20874/...,Sanofi,N/A,"Waltham, Massachusetts",External Supply Quality Control Manager(CLO ov...
...,...,...,...,...,...,...
1518,Japan Project Head,/emploi/tokyo/japan-project-head/20874/15669824,Sanofi,N/A,"Tokyo, Japon",■Responsibilities :Japan Project Lead (called ...
1519,Business Initiatives & Partnering Specialist -...,/emploi/istanbul/business-initiatives-and-part...,Sanofi,N/A,Turquie,"iMove, the Sanofi VIE Program, is available to..."
1520,Postdoctoral Fellowship: Quantitative Systems ...,/emploi/bridgewater/postdoctoral-fellowship-qu...,Sanofi,N/A,"Bridgewater, New Jersey",Discover your future…Sanofi R&D works with pas...
1521,Chemical Technician,/emploi/singapour/chemical-technician/20874/89...,Sanofi,N/A,Singapour,JOB PURPOSEAs a member of the production shift...
